In [10]:
import cv2
import torch
from PIL import Image
import timm
import numpy as np
import torchvision.transforms as transforms

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = timm.create_model("xception", pretrained=True)
model.fc = torch.nn.Linear(model.num_features, 1)  
model = model.to(device)
model.eval()

C:\Users\kevin\AppData\Roaming\Python\Python311\site-packages\timm\models\_factory.py:126: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


Xception(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act2): ReLU(inplace=True)
  (block1): Block(
    (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (rep): Sequential(
      (0): SeparableConv2d(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): SeparableConv2d(
        (conv1): Conv

In [ ]:
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  
])

In [ ]:
def analyze_video(video_path):
    cap = cv2.VideoCapture(video_path)
    fake_scores = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        frame_pil = Image.fromarray(frame_rgb)

        frame_tensor = transform(frame_pil)
        frame_tensor = frame_tensor.unsqueeze(0).to(device) 

        with torch.no_grad():
            output = model(frame_tensor)
            pred = torch.sigmoid(output.squeeze()).item()  
            fake_scores.append(pred)

    cap.release()

    avg_score = sum(fake_scores) / len(fake_scores) if fake_scores else 0
    final_decision = "Fake" if avg_score > 0.5 else "Real"

    print(f"\n🎥 Final Video Classification: {final_decision} (Avg Score: {avg_score:.4f})")
    return final_decision

In [ ]:
video_path = "ab.mp4" 
result = analyze_video(video_path)


🎥 Final Video Classification: Fake (Avg Score: 0.5408)


In [15]:
torch.save(model.state_dict(), "deepfake_model.pth")
